In [1]:
import pandas as pd

from py_helpers.db_helper import *

In [2]:
target = 'playoff'
cleaned_data = './db/WNBA_cleared.db'
tables = ['Awards_Players', 'Coaches', 'Players', 'Players_Teams', 'Teams', 'Teams_Year']

In [3]:
awards_players_df = retrieve_data(cleaned_data, tables[0])  
coaches_df = retrieve_data(cleaned_data, tables[1])  
players_df = retrieve_data(cleaned_data, tables[2])  
players_teams_df = retrieve_data(cleaned_data, tables[3])  
teams_df = retrieve_data(cleaned_data, tables[4])  
teams_year_df = retrieve_data(cleaned_data, tables[5])  


In [4]:
features_teams = ['year','tmID','rank','playoff','o_fgm','o_fga','o_ftm','o_fta','o_3pm','o_3pa','o_oreb','o_dreb','o_reb','o_asts','o_pf','o_stl','o_to','o_blk','o_pts','d_fgm','d_fga','d_ftm','d_fta','d_3pm','d_3pa','d_oreb','d_dreb','d_reb','d_asts','d_pf','d_stl','d_to','d_blk','d_pts','homeW','homeL','awayW','awayL']

# TODO: Birthdate -> BirthYear
features_player = ['bioID','pos','height','weight','birthDate'] 

features_player_teams = ['playerID','year','stint','tmID','GP','GS','minutes','points','oRebounds','dRebounds','rebounds','assists','steals','blocks','turnovers','PF','fgRatio','ftRatio','threeRatio','dq']

features_coaches = ["coachID","year","tmID","stint","won","lost"]

# TODO: add have awards to the table
features_awards_players = ['playerID','year','All-Star Game Most Valuable Player','Coach of the Year','Defensive Player of the Year', 'Kim Perrot Sportsmanship Award','Most Improved Player','Most Valuable Player','Rookie of the Year','Sixth Woman of the Year','WNBA All Decade Team Honorable Mention','WNBA All-Decade Team','WNBA Finals Most Valuable Player']
# Replace spaces with underscores in the column names
for i in range(len(features_awards_players)):
    features_awards_players[i] = features_awards_players[i].replace(' ', '_')
    features_awards_players[i] = features_awards_players[i].replace('-', '_')

# TODO: test with and without coaches dummies
features_teams_year = ['all']


In [5]:
awards_players_df = awards_players_df[features_awards_players]
coaches_df = coaches_df[features_coaches]
players_df = players_df[features_player]
players_teams_df = players_teams_df[features_player_teams]
teams_df = teams_df[features_teams]
# teams_year_df = teams_year_df[features_teams_year]

In [6]:
# Join the DataFrames
merged_df = teams_df.merge(teams_year_df, on=['year', 'tmID'])\
    .merge(players_teams_df, on=['year', 'tmID'])\
    .merge(players_df, left_on='playerID', right_on='bioID')\
    .merge(coaches_df, on=['year', 'tmID'])

# This will merge all the DataFrames based on their respective common columns.

In [7]:
merged_df = merged_df.drop(['bioID'], axis=1)
merged_df = merged_df.drop(['playerID'], axis=1)
merged_df = merged_df.drop(['coachID'], axis=1)
merged_df = merged_df.drop(['coach_none'], axis=1)

In [8]:
# Check the types of all the data
binary_check = merged_df.apply(lambda x: all(val in [0, 1] for val in x) and (x.dtype == 'int64'))
binary_columns = merged_df.columns[binary_check].tolist()
merged_df[binary_columns] = merged_df[binary_columns].astype(bool)

In [9]:
# Add the merged table to the database
table_name = 'merged'
schema = schema_builder(merged_df)
schema = f"Create Table {table_name} ({schema})"
insert_dataframe(merged_df, table_name, cleaned_data, schema)